# Coronavirus - Estimating Case Fatality Ratio

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, re, pickle

from pydemic import Pandemic, Outbreak

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

plt.rcParams['figure.figsize']=[40,20]

plt.style.use('seaborn-whitegrid')

### Setup

In [ ]:
coronavirus_confirmed_df = pd.read_csv("../data/clean/coronavirus_confirmed_global.csv", index_col=0)
coronavirus_death_df = pd.read_csv("../data/clean/coronavirus_death_global.csv", index_col=0)
coronavirus_recovered_df = pd.read_csv("../data/clean/coronavirus_recovered_global.csv", index_col=0)

In [ ]:
pandemic = Pandemic("Coronavirus", coronavirus_confirmed_df, coronavirus_death_df, coronavirus_recovered_df)
pandemic.set_smoothing_coefficient(3)
top10_countries = pandemic.get_top_regions(top_n=10, exclude=["China", "Iran"])
top20_countries = pandemic.get_top_regions(top_n=20, exclude=["United Kingdom", "Iran"])

In [ ]:
for region, outbreak in pandemic.outbreaks.items():
    outbreak.save()

In [ ]:
sars_df = pd.read_csv("../data/time_series/diff/SARS.csv", index_col=0, parse_dates=[0])
sars_outbreak = Outbreak("SARS", sars_df.Infected, sars_df.Dead)

## Simple Estimators

In [ ]:
def plot_estimator_curve(outbreak, estimator, **kwargs):
    curve = [estimator(outbreak, t) for t in range(outbreak.duration)]
    
    plt.plot(curve, label=outbreak.region, **kwargs)
    
    return 0

### Naive CFR

This is the simplest CFR estimator. It uses aggreate data of fatalities and cases at a given point in time.

In [ ]:
def nCFR(outbreak, t):
    if t > outbreak.duration:
        return np.nan
    
    return outbreak.cumulative_fatality_curve[t] / outbreak.cumulative_epidemic_curve[t]

In [ ]:
pandemic.apply(plot_estimator_curve, nCFR, regions=top10_countries)
plt.legend()
plt.show()

In [ ]:
plot_estimator_curve(sars_outbreak, nCFR)
plt.legend()
plt.show()

### Outcome-controlled Naive CFR

As seen on [2005 Guani](http://localhost:8888/notebooks/nbs/literature_review.ipynb#Methods-for-Estimating-the-Case-Fatality-Ratio-for-a-Novel,-Emerging-Infectious-Disease), this estimator will control for the censored data by only considering the resolved cases. 

A **resolved** case is one that has a known outcome of either *death* or *recovery*, whereas an **active** case is one for which no event has occured as of given time period $t$ (therefore, right-censored).

Therefore, the first assumption for this estimator is that the CFR for active cases will be similar in distribution than that of resolved cases.

Furthermore, another assumption that needs to be met for this estimator to work well is that "the hazards of death and recovery at any time $k$ from onset, conditional on an event occuring at time $k$, are proportional".

In [ ]:
def outcome_controlled_nCFR(outbreak, t):
    if t > outbreak.duration:
        return np.nan
    
    return outbreak.cumulative_fatality_curve[t] / (outbreak.cumulative_recovery_curve[t] + outbreak.cumulative_fatality_curve[t])

In [ ]:
pandemic.apply(plot_estimator_curve, outcome_controlled_nCFR, regions=top10_countries)
plt.legend()
plt.show()

### Parametric Mixture Models

As seen on [2005 Guani](http://localhost:8888/notebooks/nbs/literature_review.ipynb#Methods-for-Estimating-the-Case-Fatality-Ratio-for-a-Novel,-Emerging-Infectious-Disease). This seems powerful, yet cannot be applied to our case as we are missing patient level data.

### Underestimate-Corrected Support

In [ ]:
from scipy.optimize import fsolve
from scipy.special import gamma

# this distribution is useful as a baseline for the potential parameters we will encounter
# based on some paper
def estimate_weibull_parameters(mu=8.9, std=5.4, k_0=1):
    def optimization_problem(k):
        return (std / mu)**2 - (gamma(1+ 2/k) / gamma(1+ 1/k)) ** 2 + 1;

    # solve for k
    k = fsolve(optimization_problem, k_0)[0]   
    # solve for lambda
    l = mu / gamma(1 + 1/k) 
    
    return k, l

k, l = estimate_weibull_parameters()

In [ ]:
def known_outcome_adjusted_CFR(outbreak, t):
    case_density = outbreak.epidemic_curve[:t]
    resolved_case_rate = outbreak.resolved_case_rate[:t]
    
    result = 0
    
    for i in range(t):
        for j in range(i + 1):
            result += case_density[i - j] * resolved_case_rate[j]
            
    known_outcome_coefficients = result / case_density.sum()
    
    return nCFR(outbreak, t) * known_outcome_coefficients

In [ ]:
pandemic.apply(plot_estimator_curve, known_outcome_adjusted_CFR, regions=["France", "US", "Germany", "Belgium"])
plt.legend()
plt.show()

## Epidemic Simulation

### Distance Metrics

In [ ]:
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def dtw(x, y):
    distance, path = fastdtw(x, y, dist=euclidean)
    
    return distance

def norm(x, y):
    return np.linalg.norm(x - y)

def mean_cumulative_absolute_expected_distance(cumulative_realization, simulations):
    T = cumulative_realization.shape[0]
    cumulative_simulations = simulations.cumsum(axis=1)
    
    cumulative_absolute_expected_difference = np.abs(cumulative_realization - np.mean(cumulative_simulations, axis=0))
    
    return np.mean(cumulative_absolute_expected_difference)

### Hazard Rates

In [ ]:
from scipy.stats import weibull_min

def generate_hazard_rate(k, l, ppf_end=0.99):
    W = weibull_min(k, scale=l)
    K = int(W.ppf(ppf_end))
    
    x = np.arange(K)
    
    fatality_rate = W.pdf(x)
    cumulative_fatality_rate = W.cdf(x)
    
    hazard_rate = np.zeros(K)
    hazard_rate[0] = fatality_rate[0]
    hazard_rate[1:K] = fatality_rate[1:K] / (1 - cumulative_fatality_rate[:K-1])

    return hazard_rate

def survival_function(t, hazard_rate):
    if t > hazard_rate.shape[0]:
        raise Exception("bad arg") 
    
    return np.exp(-hazard_rate[:t].sum())

### Simulation

#### Fatality-Only Simulations

In [ ]:
def simulate_fatality_outcomes(n_cases, fatality_probability, fatality_hazard_rate):
    K = fatality_hazard_rate.shape[0]
    
    probability_samples = np.random.rand(n_cases, K + 1)
    
    fatality_bitmask = probability_samples[:, 0] < fatality_probability
    outcome_days = np.argmax(probability_samples[:, 1:] < fatality_hazard_rate, axis=1)
    
    return outcome_days, fatality_bitmask

def simulate_epidemic_fatality_outcomes(cases, fatality_probability, fatality_hazard_rate, n_sims=100):
    T = cases.shape[0]
    
    fatalities = np.zeros((n_sims, T))
    
    for t, n_cases in enumerate(cases.astype(int)):
        outcome_days, fatality_bitmask = simulate_fatality_outcomes(
            n_cases * n_sims, 
            fatality_probability, 
            fatality_hazard_rate
        )
        
        for sim in range(n_sims):
            sim_ptr = n_cases * sim
            
            sim_outcome_days = outcome_days[sim_ptr: sim_ptr + n_cases]
            sim_fatality_bitmask = fatality_bitmask[sim_ptr: sim_ptr + n_cases]
            
            sim_fatality_days = np.bincount(sim_outcome_days[sim_fatality_bitmask])
            censoring = min(sim_fatality_days.shape[0], T - t)
            
            fatalities[sim, t:(t + censoring)] += sim_fatality_days[:censoring]
            
    return fatalities

#### Dual-Event Simulations

In [ ]:
def simulate_both_outcomes(n_cases, fatality_probability, fatality_hazard_rate, recovery_hazard_rate):
    K = fatality_hazard_rate.shape[0]
    
    probability_samples = np.random.rand(n_cases, K + 1)
    
    fatality_bitmask = probability_samples[:, 0] < fatality_probability
    
    fatality_outcome_days = np.argmax(probability_samples[fatality_probability, 1:] < fatality_hazard_rate, axis=1)
    # use the inverse to make sure the two events are exclusive
    recorvey_outcome_days = np.argmax(probability_samples[~fatality_probability, 1:] > 1 - recovery_hazard_rate, axis=1)
    
    # assumption: if fatality/recovery occurs the same day, the fatality takes precendence
    # fatality_bitmask = fatality_outcomes <= recovery_outcomes
    # outcome_days = np.minimum(fatality_outcomes, recovery_outcomes)
    
    return fatality_outcome_days, recorvey_outcome_days, fatality_bitmask
    
def simulate_epidemic_both_outcomes(cases, fatality_probability, fatality_hazard_rate, recovery_hazard_rate, n_sims=100):
    T = cases.shape[0]
    K = hazard_rate.shape[0]
    
    fatalities = np.zeros((n_sims, T))
    recoveries = np.zeros((n_sims, T)) 
    
    for t, n_cases in enumerate(cases.astype(int)):
        fatality_outcome_days, recorvey_outcome_days, fatality_bitmask = simulate_cases(int(n_cases * n_sims), fatality_hazard_rate, recovery_hazard_rate)
        n_recoveries = (~fatality_bitmask).sum()
        
        for sim in range(n_sims):
            sim_outcome_days = outcome_days[n_cases * sim: n_cases * (sim + 1)]
            sim_fatality_bitmask = fatality_bitmask[n_cases * sim: n_cases * (sim + 1)]
            
            deaths_per_day = np.bincount(sim_outcome_days[sim_fatality_bitmask])
            censoring = min(deaths_per_day.shape[0], T - t)
            
            fatalities[sim, t:(t + censoring)] += deaths_per_day[:censoring]
            
    return fatalities, recoveries

## Optimization

### Bayesian Optimization

In [ ]:
from skopt import Optimizer, dump, load
from skopt.plots import plot_objective
from skopt.space import Real
from joblib import Parallel, delayed
from tabulate import tabulate
import sys

In [ ]:
class OutbreakOptimizer:
    def __init__(self, outbreak, dimensions, T=None, random_state=1, n_random_starts=20, **kwargs):
        self.optimizer = Optimizer(dimensions=dimensions, random_state=random_state, n_random_starts=n_random_starts)
        
        self.outbreak = outbreak
        self.results = {}
        
        self.T = T
        self.evaluator = self.build_evaluator(**kwargs)
        
    @staticmethod
    def from_file(region, T=None, resultpath="coronavirus"):
        return load(f"../results/{resultpath}/{region}_optimization_results_{T}.pkl")
        
    def censor(self, T, smooth=False, n_sims=1000, distance_metric=mean_cumulative_absolute_expected_distance):
        self.T = T
        
        self.evaluator = self.build_evaluator(**kwargs)
        
    def build_evaluator(self, smooth=False, n_sims=1000, distance_metric=mean_cumulative_absolute_expected_distance):
        if smooth:
            cases, real_fatalities = self.outbreak.smooth_epidemic_curve.values, self.outbreak.smooth_fatality_curve.cumsum().values
        else:
            cases, real_fatalities = self.outbreak.epidemic_curve.values, self.outbreak.fatality_curve.cumsum().values
            
        # apply censoring
        cases, real_fatalities = cases[:self.T], real_fatalities[:self.T]    
        
        def _evaluator(alpha, k, l):
            hazard_rate = generate_hazard_rate(k, l)
            fatality_simulations = simulate_epidemic_fatality_outcomes(cases, alpha, hazard_rate, n_sims)
        
            return distance_metric(real_fatalities, fatality_simulations)
        
        return _evaluator
    
    def save(self, resultpath="coronavirus"):
        dump(self.optimizer.get_result(), f"../results/{resultpath}/{self.outbreak.region}_optimization_results_{self.T}.pkl")
    
    
    def plot_results(self):
        ax = plot_objective(self.optimizer.get_result(), minimum='expected_minimum')
        ax.title(f"{self.outbreak.region} - Partial Dependence Plot - T={self.T}")
        plt.show()
        
    def optimize(self, n_runs, n_jobs=4, verbose=False):
        print(f"Parameter Optimization for {self.outbreak.region} [runs={n_runs}, workers={n_jobs}, T={self.T}]")
        
        results = []
        
        for iter_id in range(n_runs):
            suggestions = self.optimizer.ask(n_points=n_jobs)
            
            costs = Parallel(n_jobs=n_jobs)(delayed(self.evaluator)(*x) for x in suggestions)
            
            self.optimizer.tell(suggestions, costs)
            
            results += [[iter_id, worker_id, *x, costs[worker_id]] for worker_id, x in enumerate(suggestions)]
            
            if verbose:
                print(tabulate(results, headers=["Iteration", "Worker", "alpha", "k", "l", "cost"]))


### SARS

In [ ]:
for T in [None, 14, 21, 30, 60]:
    sars_opt = OutbreakOptimizer(sars_outbreak, [Real(0.00001, 0.2), Real(1.0, 4.0), Real(1.0, 20.0)], T=T)

    sars_opt.optimize(10, n_jobs=4, verbose=True)
    
    sars_opt.save()

In [ ]:
ax = plot_objective(sars_opt.optimizer.get_result(), minimum='expected_minimum')
plt.show()

In [ ]:
sars_outbreak.cumulative_fatality_curve[-1] / sars_outbreak.cumulative_epidemic_curve[-1]

#### Coronavirus

In [ ]:
for region, outbreak in pandemic.get_outbreaks(top10_countries).items():
    for T in [None, 14, 21, 30, 60]:
        opt = OutbreakOptimizer(outbreak, [Real(0.000001, 0.25), Real(1.0, 2.2), Real(10.0, 20.0)], T=T)
        opt.optimize(10, n_jobs=4)
        opt.save()

In [ ]:
plot_objective(OutbreakOptimizer.from_file("Brazil", T=14))
plot_objective(OutbreakOptimizer.from_file("Brazil", T=21))
plot_objective(OutbreakOptimizer.from_file("Brazil", T=30))
plot_objective(OutbreakOptimizer.from_file("Brazil", T=60))
plot_objective(OutbreakOptimizer.from_file("Brazil", T=None))
plt.show()

In [ ]:
plot_objective(OutbreakOptimizer.from_file("Belgium", T=14))
plot_objective(OutbreakOptimizer.from_file("Belgium", T=21))
plot_objective(OutbreakOptimizer.from_file("Belgium", T=30))
plot_objective(OutbreakOptimizer.from_file("Belgium", T=60))
plot_objective(OutbreakOptimizer.from_file("Belgium", T=None))
plt.show()

In [ ]:
def plot_best_params(region, results):
    params = results[region]["params"]
    
    h = generate_hazard_rate(params["k"], params["l"])
    fatalities = simulate_epidemic_fatality_outcomes(pandemic.outbreaks[region].smooth_epidemic_curve, params["alpha"], h)
    
    plt.title(f"{region} - Simulated deaths")
    plt.plot(pandemic.outbreaks[region].smooth_fatality_curve.values)
    plt.plot(fatalities.T, c='grey')
    plt.show()

### Optimization Results

#### First iteration

- no smoothing
- static recovery probability
- 100 simulations
- mean of mean absolute difference between simulations

In [ ]:
italy_best_parameters = {'k': 1.5680800271994926, 'l': 13.459646608653848, 'r': 0.27727495101632926}
france_best_parameters = {'k': 1.5932391692362207, 'l': 13.595494436079711, 'r': 0.2527648604733469}
spain_best_parameters = {'k': 1.156937432220853, 'l': 12.429343784489696, 'r': 0.4057687057183451}
germany_best_parameters = {'k': 1.3961487078200099, 'l': 17.922213071507144, 'r': 0.7504989403338654}
us_best_parameters = {'k': 1.680058976798015, 'l': 15.470903267327513, 'r': 0.458335669097955}

#### Second iteration

- no smoothing
- 1000 simulations
- min of mean absolute difference between simulations

# Archive

In [ ]:
import itertools
from fastprogress.fastprogress import master_bar, progress_bar

In [ ]:
r_space = np.arange(2, 30)
p_space = np.linspace(1, 0, 100, endpoint=False)
K_space = np.arange(2, 40)

### Grid Search

In [ ]:
def grid_search(model, r_space, p_space, K_space):
    r_bar = master_bar(r_space)
    p_bar = progress_bar(p_space, parent=r_bar)
    
    minimum_distance = None
    minimum_parameters = None

    for r in r_bar:
        for p in p_bar:
            for K in K_space:
                distance = model.evaluate_hazard_rate(r, p, K)

                if minimum_distance is None or minimum_distance > distance:
                    minimum_distance = distance
                    minimum_parameters = (r, p, K)

                r_bar.child.comment = f'distance={distance}'

        r_bar.main_bar.comment = f"best_params={minimum_parameters}, minimum_distance={minimum_distance}"

    return minimum_parameters, minimum_distance

### Naive CFR

Firstly, we can analyze the most recent Naive CFR values. We can use these initial values to create a search range for our estimator. We consider both the global distribution as well as the one considering only the top 10 countries.

In [ ]:
most_recent_cfr = global_coronavirus_outbreak.cfr_curve.iloc[-1]

pd.DataFrame({"World": most_recent_cfr.describe(), "Top10": most_recent_cfr[coronavirus_top10_countries].describe()})

As we can see above, using all countries greatly reduces the mean and quartile values of CFR. This makes sense as a lot of countries are yet to hit their peak of the infection, and so their current CFR is an underestimation of what it will eventually reach (mostly due to the lag between diagnosis and outcome + the pile-on effects of exhausted health service resources).

#### First-death-adjusted CFR

We can notice that the Naive CFR estimator varies a lot in the early stages and begins to steadily grow as the epidemic develops (towards a flatenning at different levels).

In [ ]:
# fetch first death time lags from global death df
first_death_time_lags = global_death_df.eq(0).sum().apply(lambda x: pd.Timedelta(days=x))

ax = plt.gca()
k=1

for country in coronavirus_top10_countries:
    cumulative_country_df = global_coronavirus_outbreak.get_region_df(country).cumsum(axis=0)
    
    first_death_in_country = first_death_time_lags[country]
    corrected_cumulative_country_df = cumulative_country_df.loc[first_death_in_country:, :].reset_index(drop=True)
    
#     (corrected_country_df.loc[:, "Dead"].cumsum() / corrected_country_df.loc[:, "Infected"].cumsum()).rolling(k).mean().plot(ax=ax, label=country)

    (corrected_cumulative_country_df["Dead"] / corrected_cumulative_country_df["Infected"]).rolling(k).mean().plot(ax=ax, label=country)
    
plt.legend()

ax.set_xlabel("CFR")
ax.set_xlabel("days since first death")

global_coronavirus_outbreak.add_plot_details(ax, f"Naive CFR - MA={k} days")

### Individual epidemic study

First, we want to convert our indices to time deltas to facilitate manipulation.

In [ ]:
# find first day that confirm cases aren't 0 for every country
first_confirmed_case_time_lags = global_confirmed_df.eq(0).sum().apply(lambda x: pd.Timedelta(days=x))

In [ ]:
for country in global_coronavirus_outbreak.regions:
    country_df = global_coronavirus_outbreak.get_region_df(country)
    
    first_confirmed_case_in_country = first_confirmed_case_time_lags[country]
    corrected_country_df = country_df.loc[first_confirmed_case_in_country - pd.Timedelta(days=1):, :]
    
    corrected_country_df.loc[:, "CFR"] = corrected_country_df.loc[:, "Dead"] / corrected_country_df.loc[:, "Infected"]
    
    corrected_country_df.to_csv(f"../data/countries/{country}.csv", index=None)

### US

In [ ]:
us_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv') \
                    .drop(["UID", "iso2", "iso3", "code3", "Admin2", "Lat", "Long_", "FIPS", "Country_Region", "Combined_Key"], axis=1).groupby('Province_State').sum().transpose()
us_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')\
                .drop(["UID", "iso2", "iso3", "code3", "Admin2", "Lat", "Long_", "FIPS", "Country_Region", "Combined_Key", "Population"], axis=1).groupby('Province_State').sum().transpose()

In [ ]:
us_coronavirus_outbreak = Outbreak("us_coronavirus")

us_coronavirus_outbreak.set_epidemic_curve(us_confirmed_df)
us_coronavirus_outbreak.set_fatality_curve(us_death_df)

us_coronavirus_outbreak.convert_indices_to_timedelta_since_epidemic_start_date()

In [ ]:
us_coronavirus_outbreak.filter_top_regions()

In [ ]:
top_regions_total_number_of_cases = us_coronavirus_outbreak.epidemic_curve.sum(axis=1)[-1]
us_total_number_of_cases = us_coronavirus_outbreak.base_epidemic_curve.sum(axis=1)[-1]

top_regions_total_number_of_deaths = us_coronavirus_outbreak.fatality_curve.sum(axis=1)[-1]
us_total_number_of_deaths = us_coronavirus_outbreak.base_fatality_curve.sum(axis=1)[-1]

print("Case coverage=%.2f" % ((top_regions_total_number_of_cases / us_total_number_of_cases) * 100))
print("Death coverage=%.2f" % ((top_regions_total_number_of_deaths / us_total_number_of_deaths) * 100))

We should remove those regions not yet affected by the pandemic.

In [ ]:
ax = plt.gca()

us_coronavirus_outbreak.cfr_curve.fillna(0).plot(ylim=[-0.01, 0.1], ax=ax)

us_coronavirus_outbreak.add_plot_details(ax, "cfr_curve")